In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002357959747314453
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 27.55115870652108
200000 44.81890659412297
300000 62.98270479774479
400000 55.99539322604793
500000 57.600721212488246
600000 58.82411447528189
700000 56.897808840705586
800000 57.47900187385066
900000 54.78990770487572
1000000 57.34909376368021
1100000 38.88546831088563
1200000 32.43097184295923
1300000 56.49598959086313
1400000 53.930580454864085
1500000 55.88522616851801
1600000 54.946071159465184
1700000 57.69991047766511
1800000 56.18346047842321
1900000 54.245567469009835
2000000 57.680706484281885
2100000 54.63599630651943
2200000 43.869252585694696
2300000 29.16256537981622
2400000 53.669569969636015
2500000 56.491110359896915
2600000 51.75842065482744
2700000 55.10472745718729
2800000 56.70899058150976
2900000 53.14787953982029
3000000 54.482491

29700000 19.827338384893004
29800000 52.121813129635875
29900000 63.90001046045297
30000000 60.917198596571495
30100000 61.9821781658085
30200000 63.711139444661804
30300000 60.17992786643382
30400000 70.37324139605659
30500000 63.35620410578464
30600000 24.047163848915268
30700000 66.46397881481123
30800000 65.78430280563568
30900000 68.12133773772818
31000000 38.64797577176601
31100000 39.91030603983323
31200000 62.13854297030259
31300000 58.108225160267715
31400000 67.41754341391749
31500000 57.805429283245964
31600000 63.07587341839381
31700000 59.30548796173976
31800000 55.3614920577133
31900000 18.163687829031222
32000000 50.91233902650882
32100000 62.785320979542156
32200000 58.330176641747066
32300000 59.986120854039854
32400000 65.801141936316
32500000 56.773419874891935
32600000 68.08230710875212
32700000 51.889809967289565
32800000 30.675503972689548
32900000 64.30813763198469
33000000 62.51320052466761
33100000 69.37541050912718
33200000 44.042247173171596
33300000 31.68181

59800000 34.161539689923
59900000 44.262374322145824
60000000 59.84194822307032
60100000 56.630430976554536
60200000 51.8670788013922
60300000 56.975637463050134
60400000 41.716797222084324
60500000 22.437206585820448
60600000 51.9470739903059
60700000 52.04147644679912
60800000 55.694092582522316
60900000 51.79773351602138
61000000 61.65835540521701
61100000 24.469139568625845
61200000 55.50343575477967
61300000 64.09515062543178
61400000 56.232782945033236
61500000 64.7663182792328
61600000 53.99752487672386
61700000 53.9847199328099
61800000 51.47004228277185
61900000 50.26758576341297
62000000 35.523367106058046
62100000 36.67472687661978
62200000 49.38698084549331
62300000 45.52850590205924
62400000 42.582082141180635
62500000 47.084083788464284
62600000 37.715243551526314
62700000 23.205417407825667
62800000 45.10499164962092
62900000 44.47505003507414
63000000 45.474741562414366
63100000 44.10392341137061
63200000 51.80531637440743
63300000 21.74457942619623
63400000 50.06077590

89900000 48.144731067716634
90000000 47.32837312540143
90100000 48.53543410939369
90200000 47.97118133886135
90300000 46.487927329178135
90400000 49.785627365691084
90500000 48.93417042297764
90600000 44.85666874248418
90700000 43.899407456039164
90800000 24.74104896192482
90900000 36.57567295738372
91000000 46.09474449556778
91100000 43.616608758123924
91200000 24.492670955655225
91300000 38.38581035632698
91400000 44.4042417413828
91500000 44.2414132739595
91600000 28.47902353517301
91700000 36.25586696771006
91800000 43.70881908839257
91900000 47.478842494680116
92000000 47.613429597142286
92100000 51.35133376543564
92200000 47.50068266458235
92300000 48.82745967021977
92400000 49.42360341912187
92500000 44.81767106089972
92600000 52.05674668447003
92700000 46.65347548484285
92800000 43.42549512171979
92900000 42.52568337317697
93000000 29.612599310325464
93100000 34.14698199116736
93200000 41.19807091717756
93300000 42.1885336902576
93400000 22.551093591265534
93500000 36.132520742

119300000 54.13847254765149
119400000 44.376864789832226
119500000 51.369647371938896
119600000 16.37143770590981
119700000 45.521725336476166
119800000 15.141838244469179
119900000 44.95157471314887
120000000 22.373446745088525
120100000 53.016499261056126
120200000 43.67591842631558
120300000 54.95160418650429
120400000 44.475459633839385
120500000 52.57132447474474
120600000 48.7585757468464
120700000 51.73692257232888
120800000 60.09552934882234
120900000 56.184390380992404
121000000 54.53047731021738
121100000 51.77882656820137
121200000 45.24486642194901
121300000 53.24214720977755
121400000 43.49754224395019
121500000 52.39205028987678
121600000 48.48200878833143
121700000 45.60249647032953
121800000 23.707413693611258
121900000 41.442706987316996
122000000 12.788467887835752
122100000 37.7046969060881
122200000 29.03044584402339
122300000 46.96073735506254
122400000 47.14286463348453
122500000 52.46731275415068
122600000 47.89015875372833
122700000 48.11027485623302
122800000 5

...

638000000 2.2200782332934024
638100000 1.8604312084104353
638200000 1.1477317596914536
638300000 1.453285991193855
638400000 1.63184707568784
638500000 1.3536001950345247
638600000 2.0330205135382298
638700000 1.443504297757829
638800000 1.3507013615405845
638900000 1.3331236137046396
639000000 1.8886706169167353
639100000 1.4708887891662863
639200000 1.2644427685188115
639300000 2.006989456333317
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.9188437210876064
639500000 2.4005834143514013
639600000 2.1236548750974187
639700000 0.9457816582775269
639800000 1.0427115230073989
639900000 0.8558012804809054
640000000 0.9190946795039726
640100000 0.9294718429691633
640200000 0.9288847970961452
640300000 1.0677516304986172
640400000 1.008496963904661
640500000 1.594112098567656
640600000 1.6831416450537404
640700000 1.233276996227932
640800000 0.9415900946962807
640900000 1.043660722955382
641000000 0.8660675958151599
641100000 0.9051822099577017
641200000 0

664800000 2.051656162171637
664900000 2.0422878838261416
665000000 1.7520041731145084
665100000 2.2477137238168
665200000 2.131824337283693
665300000 1.348798084322224
665400000 0.34295208948031664
665500000 0.3430105464748462
665600000 0.3431059910307864
665700000 0.3436064186088754
665800000 0.3424307401497728
665900000 0.5442982401269137
666000000 0.5840901473479952
666100000 0.5563432915739926
666200000 0.3328372242183586
666300000 0.33163050951781003
666400000 1.6485298149101246
666500000 2.3701905027458174
666600000 2.7282934972993673
666700000 1.9818259633467894
666800000 2.9358331610283313
666900000 1.9397738316892135
667000000 1.9128632000658352
667100000 2.0650780493012824
667200000 1.9598121748640756
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.7901026135167797
667400000 0.2944858763806979
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.4436108689110312
667600000 1.7174449571541488
667700000 1.3751843126

690600000 1.302474174029418
690700000 0.9116909543198526
690800000 1.7640330424427986
690900000 1.8458863842311006
691000000 1.8097060613623832
691100000 1.3448503798266827
691200000 0.4973874762310551
691300000 1.2515132593873806
691400000 1.3889196463330844
691500000 1.4861160657351904
691600000 1.2359575773571605
691700000 0.9056481472951412
691800000 0.9671459275786798
691900000 1.211801155115245
692000000 1.288524005114949
692100000 1.3145573716516346
692200000 1.4004927166109518
692300000 0.612245605266728
692400000 1.0523698971441957
692500000 1.2934292076886527
692600000 1.8225809905490198
692700000 1.2219868324564165
692800000 0.8941047947446061
692900000 0.6463198621089756
693000000 1.2700554695917188
693100000 1.352424927285499
693200000 1.4365821699338477
693300000 1.0495249254144132
693400000 0.9049881955233502
693500000 0.9658476684386909
693600000 1.2888414351909518
693700000 1.5106811262898312
693800000 1.21763687160936
693900000 0.8950642165864249
694000000 0.726034426

717900000 0.11067241479974853
718000000 0.11027755532868902
718100000 0.10976282335991264
718200000 0.10984061866376599
718300000 0.258074346381902
718400000 0.32533261343073644
718500000 0.2805870485345006
718600000 0.33449683640660116
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.31729373153208495
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.09585037148844136
718900000 0.10667456545633341
719000000 0.1065681770304203
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.281701929181644
719200000 0.448612954164505
719300000 0.4370106774549127
719400000 0.39677540781866183
719500000 0.3895302435495251
719600000 0.4051904026122491
719700000 0.3861320358359648
719800000 0.37370314222134676
719900000 0.3486196503449381
720000000 0.4174334350752645
720100000 0

738100000 0.19026183449935316
738200000 0.15909421194236875
738300000 0.19526364413433406
738400000 0.26504208275752733
738500000 0.16537247160112065
738600000 0.20789235761301847
738700000 0.1582213209053278
738800000 0.18411977126311063
738900000 0.17720607011969222
739000000 0.14231852141380574
739100000 0.15946459672114252
739200000 0.17118740668721927
739300000 0.14129907117540041
739400000 0.16728978790822624
739500000 0.19318867744513815
739600000 0.14924436709798508
739700000 0.15249909787493346
739800000 0.17218157362954484
739900000 0.17863797371033366
740000000 0.17608656027544736
740100000 0.15292249967877003
740200000 0.1761884815758513
740300000 0.15279087471118172
740400000 0.13416538974113132
740500000 0.1288596218644407
740600000 0.15843040143622758
740700000 0.13224640376319688
740800000 0.1252996290061169
740900000 0.13495169364882906
741000000 0.15421148246962427
741100000 0.1368199720191651
741200000 0.21117507827574614
741300000 0.15106136802641748
741400000 0.152